In [49]:
import numpy as np
import pandas as pd
import matplotlib as mlt

#raw data from PDF via tabula
raw = pd.read_csv('tabula-death_penalty.csv')

#delete instances of multiple headers
#(there are a lot in the raw data)
df = raw[raw['SEX'] != "SEX"]

#print the data without repeated headers 
no_headers = df.to_csv('death_penalty_no_headers.csv')

#load in web database output:
#1967—present
edb = pd.read_csv('execution_database.csv')
espy = pd.read_csv('death_penalty_no_headers.csv')


In [50]:
#load in keys for in which states death 
#penalty is legal
legal = pd.read_csv('legal.csv')

edb['Date'] = pd.to_datetime(edb['Date'])

#create new DF to cleanly concat two 
#datasets
edb_new = pd.DataFrame()

edb_new['NAME (Last First)'] = edb['Name']
edb_new['RACE'] = edb['Race']
edb_new['AGE'] = edb['Age']
edb_new['SEX'] = edb['Sex']
edb_new['CRIME'] = 'Missing'
edb_new['METHOD'] = edb['Method']
edb_new['DAY'] = edb['Date'].dt.day
edb_new['MONTH'] = edb['Date'].dt.month
edb_new['YEAR'] = edb['Date'].dt.year
edb_new['STATE'] = edb['State']
edb_new['DATE'] = edb['Date']


mask = (edb_new['DATE'] > '12/31/2002')

#make sure data from each for 2002 
#is the same so dfs don't overlap
#before concat

mask2002 = (edb_new['YEAR'] == 2002)
mask22002 = (espy['YEAR'] == 2002)

two002 = edb_new.loc[mask2002]
two2002 = espy.loc[mask22002]

In [51]:
edb_new = edb_new.loc[mask]

frames = [espy, edb_new]

result = pd.concat(frames)

result['executed'] = 'y'
result['METHOD'].fillna(value='Missing')

result.to_csv('concatenated.csv')

In [52]:
methodclean = result.loc[result['METHOD'] == '']
methodclean.to_csv('method.csv')

In [53]:
print 'Number of men and women executed in the 19th Century:' 

masknineteenth = (espy['YEAR'] >= 1800) & (espy['YEAR'] < 1900)

nineteenth = espy.loc[masknineteenth]

print nineteenth['SEX'].value_counts()
print nineteenth['RACE'].value_counts()

nineteenth.to_csv('nineteenth.csv')

Number of men and women executed in the 19th Century:
M    5078
F     175
Name: SEX, dtype: int64
Black                 2559
White                 2048
unknown                332
Native Amer.           253
Hispanic               115
Asian - Pacific Is      36
Name: RACE, dtype: int64


In [54]:
pivot = result.pivot_table(index=['STATE'],values='executed',aggfunc='count')

#new df from pivot table 
statecounts = pd.DataFrame(pivot)

statecounts.reset_index(inplace=True)

new = pd.merge(statecounts, legal, on='STATE')

print new.head(2)


  STATE  executed Death Penalty Legal? 
0    AK        12                    no
1    AL       757                   yes


In [55]:
pivot1 = new.pivot_table(index=['Death Penalty Legal? '],values='executed',aggfunc='sum')
print pivot1

Death Penalty Legal? 
m       1368
no      3343
yes    11136
Name: executed, dtype: int64


In [56]:
crimes = result['CRIME'].value_counts()
#crimes.to_csv('crimes.csv')

methods = result['METHOD'].value_counts()
methods.to_csv('methods.csv')

cleanedcrimes = pd.read_csv('cleaned_crimes.csv')
cleanedmethods = pd.read_csv('methodcleaning.csv')

result = pd.merge(result, cleanedcrimes, on="CRIME", how='outer')
result = pd.merge(result, cleanedmethods, on="METHOD", how='outer')

result.to_csv('clean.csv')